In [1]:
%load_ext autoreload
%autoreload 2
#from lib.multitasking_lib import *

In [2]:
from lib.multitasking_lib import *
cache_type ='backtest_uni'

my_cache = cache_state('backtest_uni')

In [3]:
from sqlalchemy import Column, Integer, String, create_engine, select
from sqlalchemy.orm import sessionmaker
from werkzeug.security import generate_password_hash

import os
database_url = os.environ.get('DATABASE_URL')

engine = create_engine(database_url, echo=False)
Session = sessionmaker(bind=engine)


In [4]:
session = Session()

#conn = session.bind

######################
# Initiate Backtest  #
######################
def store_algo(algo, algo_name="default"):
    redis_conn.hset("algos",algo_name, algo)
    algo_f = open("algo/"+algo_name+".txt", "w")
    algo_f.write(algo)
    algo_f.close()

def load_algo_to_redis(filename):
    algo = open('algo/'+filename+'.txt')
    algo_str = algo.read()
    my_cache.hset('algos', filename,algo_str)
    
#my_cache.flushall()
def backtest_unitest(stocks, freq='1T',algo_name = 'haikin_1_new', fromDate='2019-4-1', toDate='2019-4-2', mode='full'):
    stocks = stocks
    sl = 1
    target = 1
    qty = 10
    job_id = "{}-{}".format(int(dt.now().timestamp()*1000000),1) 
    backtest_msg={'job_id':job_id,'stock':stocks,'sl':sl,'target':target,'qty':qty,'algo':algo_name,'fromDate':fromDate,'toDate':toDate,'freq':freq, 'mode':mode}
    
    load_algo_to_redis(algo_name)
    
    # Step 4: Done is set to 0: Backtest is in progress, will be resetted by backtest job
    redis_conn.set('done'+cache_type,0)
    # Step 5: Send the msg to backtest thread to initiate the back test
    #pdebug(json.dumps(backtest_msg))
    user = session.query(User).filter(User.id==1).first()
    user.jobs.append(Jobs(job_id=job_id, job_type='backtest_'+mode, job_status='INIT', job_info=json.dumps(backtest_msg)))
    session.commit()
    redis_conn.publish('kite_simulator'+cache_type,json.dumps(backtest_msg))
    
##########################
# Unit Test              #
##########################

open('log/freedom.log','r+')
#log_file.close()

logger.setLevel(10)
loggerT.setLevel(21)
my_cache.set('done'+cache_type,1)
my_cache.xtrim('msgBufferQueue'+cache_type,0,False)
my_cache.set('last_id_msg'+cache_type, 0)


try:
    if backtest_manager.job.is_alive():
        backtest_manager.job.terminate()
        time.sleep(0.1)
except:
    pass
finally:
    backtest_manager = threadManager(cache_type, ["kite_simulator","ohlc_tick_handler","order_handler"], [kite_simulator, ohlc_tick_handler, order_handler])

#redis_conn.publish('ohlc_tick_handler'+cache_type,'start')
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
print(jobs)


2020-06-08 00:03:14,129 INFO:	Starting Handler: kite_simulator
2020-06-08 00:03:14,135 INFO:	Starting Handler: ohlc_tick_handler


True
['order_notification_handlerlive', 'ohlc_tick_handlerbacktest_web', 'order_handlerlive', 'kite_simulatorbacktest_web', 'order_handlerbacktest_web', 'ohlc_tick_handlerlive']
[<Process(Process-1, started)>]


2020-06-08 00:03:14,141 INFO:	Starting Handler: order_handler
2020-06-08 00:03:14,207 DEBUG:	order_handler(backtest_uni): 1
2020-06-08 00:03:14,164 DEBUG:	ohlc_tick_handler(backtest_uni) - INIT: 1
2020-06-08 00:03:14,153 DEBUG:	kite_simulator: 1
2020-06-08 00:03:14,236 ERROR:	Error in order handler
2020-06-08 00:03:14,297 ERROR:	kite_simulator: Invalid msg: 1
2020-06-08 00:03:15,481 DEBUG:	kite_simulator: {"job_id": "1591554794400630-1", "stock": ["TCS", "WIPRO"], "sl": 1, "target": 1, "qty": 10, "algo": "haikin_1_new", "fromDate": "2019-4-1", "toDate": "2019-4-2", "freq": "1T", "mode": "full"}
2020-06-08 00:03:15,496 DEBUG:	Trade_init: TCS
2020-06-08 00:03:15,522 INFO:	Reset Cache for: TCS1backtest_uni
2020-06-08 00:03:16,485 INFO:	TCS1backtest_uni=>{'stock': '', 'qty': '0', 'SL %': '0.0', 'TP %': '0.0', 'amount': '0', 'price': '0.0', 'P&L': '0.0', 'P&L %': '0.0', 'Total P&L': '0.0', 'Total P&L %': '0.0', 'low': '0.0', 'sl': '0.0', 'ltp': '0.0', 'ltp %': '0.0', 'tp': '0.0', 'high': '0

2020-06-08 00:03:33,215 ERROR:	Issue in algotrade
TRADE: B: EN:  : 2019-04-01 09:39 : TCS -> 2013.5
2020-06-08 00:03:33,928 DEBUG:	Place Order: 2013.5,0.0,0,-43.49999999999682
2020-06-08 00:03:33,976 DEBUG:	order_handler(backtest_uni): {"cmd": "buy", "symbol": "TCS1", "price": 2013.5, "qty": 10.0}
2020-06-08 00:03:34,010 INFO:	Placeorder(full):buy: TCS: 10x2013.5
2020-06-08 00:03:34,034 INFO:	PO:LONG
TRADE: S: EX:  : 2019-04-01 09:44 : TCS -> 2016.05
2020-06-08 00:03:35,830 DEBUG:	Place Order: 2016.05,2013.5,0,-43.49999999999682
2020-06-08 00:03:35,850 DEBUG:	order_handler(backtest_uni): {"cmd": "sell", "symbol": "TCS1", "price": 2016.05, "qty": 10.0}
2020-06-08 00:03:35,875 INFO:	Placeorder(full):sell: TCS: 10x2016.05
2020-06-08 00:03:35,900 INFO:	SQUAREOFF:LONG
2020-06-08 00:03:35,964 ERROR:	Issue in algotrade
TRADE: B: EX:  : 2019-04-01 09:44 : WIPRO -> 257.2
2020-06-08 00:03:36,129 DEBUG:	Place Order: 257.2,257.5,0,-5.999999999999659
2020-06-08 00:03:36,159 DEBUG:	order_handler(bac

TRADE: S: EN:  : 2019-04-01 10:15 : WIPRO -> 260.7
2020-06-08 00:03:50,429 DEBUG:	Place Order: 260.7,0.0,0,16.50000000000034
2020-06-08 00:03:50,450 DEBUG:	order_handler(backtest_uni): {"cmd": "sell", "symbol": "WIPRO1", "price": 260.7, "qty": 10.0}
2020-06-08 00:03:50,477 INFO:	Placeorder(full):sell: WIPRO: 10x260.7
2020-06-08 00:03:50,512 INFO:	PO:SHORT
TRADE: B: EN:  : 2019-04-01 10:17 : TCS -> 2027.5
2020-06-08 00:03:51,212 DEBUG:	Place Order: 2027.5,0.0,0,63.00000000000409
2020-06-08 00:03:51,230 DEBUG:	order_handler(backtest_uni): {"cmd": "buy", "symbol": "TCS1", "price": 2027.5, "qty": 10.0}
2020-06-08 00:03:51,252 INFO:	Placeorder(full):buy: TCS: 10x2027.5
2020-06-08 00:03:51,281 INFO:	PO:LONG
TRADE: B: EX:  : 2019-04-01 10:20 : WIPRO -> 260.65
2020-06-08 00:03:53,082 DEBUG:	Place Order: 260.65,260.7,0,16.50000000000034
2020-06-08 00:03:53,103 DEBUG:	order_handler(backtest_uni): {"cmd": "buy", "symbol": "WIPRO1", "price": 260.65, "qty": 10.0}
2020-06-08 00:03:53,149 INFO:	Place

...

2020-06-08 00:05:14,101 INFO:	Placeorder(full):buy: TCS: 10x2028.4
2020-06-08 00:05:14,135 INFO:	PO:LONG
TRADE: S: EX:  : 2019-04-01 12:54 : TCS -> 2028.15
2020-06-08 00:05:14,604 DEBUG:	Place Order: 2028.15,2028.4,0,55.000000000009095
2020-06-08 00:05:14,625 DEBUG:	order_handler(backtest_uni): {"cmd": "sell", "symbol": "TCS1", "price": 2028.15, "qty": 10.0}
2020-06-08 00:05:14,727 INFO:	Placeorder(full):sell: TCS: 10x2028.15
2020-06-08 00:05:14,760 ERROR:	Issue in algotrade
2020-06-08 00:05:14,779 INFO:	SQUAREOFF:LONG
TRADE: S: EN:  : 2019-04-01 12:54 : WIPRO -> 261.9
2020-06-08 00:05:15,017 DEBUG:	Place Order: 261.9,0.0,0,26.499999999998636
2020-06-08 00:05:15,034 DEBUG:	order_handler(backtest_uni): {"cmd": "sell", "symbol": "WIPRO1", "price": 261.9, "qty": 10.0}
2020-06-08 00:05:15,067 INFO:	Placeorder(full):sell: WIPRO: 10x261.9
2020-06-08 00:05:15,109 INFO:	PO:SHORT
TRADE: B: EX:  : 2019-04-01 12:55 : WIPRO -> 261.9
2020-06-08 00:05:15,717 DEBUG:	Place Order: 261.9,261.9,0,26.4999

2020-06-08 00:05:28,385 ERROR:	Issue in algotrade
TRADE: S: EX:  : 2019-04-01 13:16 : TCS -> 2028.75
2020-06-08 00:05:29,045 DEBUG:	Place Order: 2028.75,2030.5,0,34.000000000010004
2020-06-08 00:05:29,078 DEBUG:	order_handler(backtest_uni): {"cmd": "sell", "symbol": "TCS1", "price": 2028.75, "qty": 10.0}
2020-06-08 00:05:29,110 INFO:	Placeorder(full):sell: TCS: 10x2028.75
2020-06-08 00:05:29,144 ERROR:	Issue in algotrade
2020-06-08 00:05:29,148 INFO:	SQUAREOFF:LONG
TRADE: S: EX:  : 2019-04-01 13:16 : WIPRO -> 262.15
2020-06-08 00:05:29,421 DEBUG:	Place Order: 262.15,262.25,0,27.99999999999841
2020-06-08 00:05:29,432 DEBUG:	order_handler(backtest_uni): {"cmd": "sell", "symbol": "WIPRO1", "price": 262.15, "qty": 10.0}
2020-06-08 00:05:29,461 INFO:	Placeorder(full):sell: WIPRO: 10x262.15
2020-06-08 00:05:29,504 INFO:	SQUAREOFF:LONG
TRADE: B: EN:  : 2019-04-01 13:18 : TCS -> 2028.5
2020-06-08 00:05:30,158 DEBUG:	Place Order: 2028.5,0.0,0,16.500000000010004
2020-06-08 00:05:30,179 DEBUG:	or

2020-06-08 00:05:43,740 DEBUG:	Place Order: 262.5,0.0,0,27.499999999998295
2020-06-08 00:05:43,766 DEBUG:	order_handler(backtest_uni): {"cmd": "buy", "symbol": "WIPRO1", "price": 262.5, "qty": 10.0}
2020-06-08 00:05:43,793 INFO:	Placeorder(full):buy: WIPRO: 10x262.5
2020-06-08 00:05:43,819 INFO:	PO:LONG
2020-06-08 00:05:43,923 ERROR:	Issue in algotrade
TRADE: S: EX:  : 2019-04-01 13:44 : TCS -> 2028.85
2020-06-08 00:05:44,447 DEBUG:	Place Order: 2028.85,2028.9,0,12.000000000007276
2020-06-08 00:05:44,476 DEBUG:	order_handler(backtest_uni): {"cmd": "sell", "symbol": "TCS1", "price": 2028.85, "qty": 10.0}
2020-06-08 00:05:44,491 INFO:	Placeorder(full):sell: TCS: 10x2028.85
2020-06-08 00:05:44,515 INFO:	SQUAREOFF:LONG
2020-06-08 00:05:44,573 ERROR:	Issue in algotrade
TRADE: S: EX:  : 2019-04-01 13:44 : WIPRO -> 262.35
2020-06-08 00:05:44,844 DEBUG:	Place Order: 262.35,262.5,0,27.499999999998295
2020-06-08 00:05:44,860 DEBUG:	order_handler(backtest_uni): {"cmd": "sell", "symbol": "WIPRO1",

2020-06-08 00:06:01,924 DEBUG:	order_handler(backtest_uni): {"cmd": "sell", "symbol": "TCS1", "price": 2029.2, "qty": 10.0}
2020-06-08 00:06:01,971 INFO:	Placeorder(full):sell: TCS: 10x2029.2
2020-06-08 00:06:02,002 INFO:	SQUAREOFF:LONG
2020-06-08 00:06:02,018 ERROR:	Issue in algotrade
TRADE: B: EX:  : 2019-04-01 14:21 : WIPRO -> 262.45
2020-06-08 00:06:02,214 DEBUG:	Place Order: 262.45,262.55,0,20.999999999998522
2020-06-08 00:06:02,256 DEBUG:	order_handler(backtest_uni): {"cmd": "buy", "symbol": "WIPRO1", "price": 262.45, "qty": 10.0}
2020-06-08 00:06:02,289 INFO:	Placeorder(full):buy: WIPRO: 10x262.45
2020-06-08 00:06:02,324 INFO:	SQUAREOFF:SHORT
2020-06-08 00:06:02,344 ERROR:	Issue in algotrade
TRADE: B: EN:  : 2019-04-01 14:22 : TCS -> 2027.95
2020-06-08 00:06:02,526 DEBUG:	Place Order: 2027.95,0.0,0,42.000000000007276
2020-06-08 00:06:02,541 DEBUG:	order_handler(backtest_uni): {"cmd": "buy", "symbol": "TCS1", "price": 2027.95, "qty": 10.0}
2020-06-08 00:06:02,569 INFO:	Placeorder

2020-06-08 00:06:16,488 INFO:	Placeorder(full):buy: WIPRO: 10x263.25
2020-06-08 00:06:16,514 INFO:	SQUAREOFF:SHORT
2020-06-08 00:06:16,561 ERROR:	Issue in algotrade
2020-06-08 00:06:28,166 INFO:	Kite_Simulator: Done: 375


In [5]:
backtest_unitest(['TCS','WIPRO'], algo_name='haikin_1_new')

In [9]:
%%time
my_cache.getTrades('TCS1')

AttributeError: 'NoneType' object has no attribute 'id'

In [10]:
my_cache.getValue()

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,tp,high,last_processed,state,mode,algo,freq,hdf_freq,order_id,job_id
0,TCS,10,1,1,20371.5,0,-18.49999999999909,-0.9073075036782291,33.00000000000409,0.1619910168618123,...,0,2038.0,1554112740.0,SCANNING,full,haikin_1_new,1T,minute,0,1591554794400630-1
1,WIPRO,10,1,1,2632.5,0,1.0000000000002274,0.3797228023543677,18.999999999999204,0.7217473884140249,...,0,262.8,1554112740.0,SCANNING,full,haikin_1_new,1T,minute,0,1591554794400630-1


In [14]:
################################
###         Stop Trade       ###
################################
pinfo(cache_type)
my_cache.xtrim('msgBufferQueue'+cache_type,0,False)
my_cache.delete('msgBufferQueue'+cache_type)
my_cache.set('TCSbacktest_uniTICK',pd.DataFrame().to_json())
session.rollback()
if backtest_manager.job.is_alive():
    backtest_manager.job.terminate()
time.sleep(1)
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
my_cache.set('last_id_msg',0)
jobs
jobs = []

2020-06-07 23:13:03,518 INFO:	backtest_uni


False
['order_notification_handlerlive', 'ohlc_tick_handlerbacktest_web', 'order_handlerlive', 'kite_simulatorbacktest_web', 'order_handlerbacktest_web', 'ohlc_tick_handlerlive']


In [10]:
for key in my_cache.keys('*backtest_uni*'):
    my_cache.delete(key)

In [ ]:
my_cache.publish('kite_simulatorbacktest_web','stop')

In [ ]:
my_cache.pubsub_channels()

In [ ]:
backtest_unitest(['TCS'])

In [ ]:
backtest_unitest(['WIPRO','TCS'])

In [36]:
from sqlalchemy import and_
conn = session.bind
web_cache = cache_state('backtest_web')
job_id = web_cache.getValue('TCS1','job_id')
stock = web_cache.getValue('TCS1','stock')
job = session.query(Jobs).filter(Jobs.job_id==job_id).first().id

trades = session.query(Trades).filter(Trades.job_id==job)

tmp_df = pd.read_sql(select([Trades]).where(and_(Trades.job_id==job, Trades.stock==stock)), conn)

tmp_df["buy"] = tmp_df[tmp_df['buy_or_sell']=='B'].price
tmp_df["sell"] = tmp_df[tmp_df['buy_or_sell']=='S'].price
tmp_df1 = tmp_df.drop(columns=['id', 'stock', 'price', 'qty', 'buy_or_sell',
       'order_id', 'job_id'])
tmp_df1.columns = ['date','mode','buy','sell']

tmp_df1 = tmp_df1.set_index('date')
tmp_df1.index = pd.to_datetime(tmp_df1.index)

In [37]:
tmp_df1

,mode,buy,sell
date,,,
2019-04-23 09:17:00,B: EN:,2154.05,NaN
2019-04-23 09:18:00,S: EX:,NaN,2147.35
2019-04-23 09:20:00,B: EN:,2149.30,NaN
2019-04-23 09:21:00,S: EX:,NaN,2149.30
2019-04-23 09:24:00,B: EN:,2143.75,NaN
...,...,...,...
2019-04-23 14:44:00,S: EX:,NaN,2149.50
2019-04-23 14:45:00,B: EN:,2149.20,NaN
2019-04-23 14:46:00,S: EX:,NaN,2151.25


In [27]:
#web_cache = cache_state('backtest_web')
trades_df = web_cache.getTrades('TCS1')

trades_df[trades_df['buy'].isna()==False]['buy']

[autoreload of lib.data_model_lib failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/usr/local/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/usr/local/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 292, in update_class
    if (old_obj == new_obj) is True:
  File "/usr/local/lib/python3.7/site-packages/sqlalchemy/sql/operators.py", line 365, in __eq__
    return self.operate(eq, other)
  File "/usr/local/lib/python3.7/site-packages/sqlalchemy/orm/attributes.py", line 218, in operate
    return op(self.comparator, *other, **kwargs)
  File "/usr/local/lib/python3.7/site-packages/sqlalchemy/orm/relationships.py", 

date
2019-04-23 09:17    2154.05
2019-04-23 09:20    2149.30
2019-04-23 09:24    2143.75
2019-04-23 09:26    2145.55
2019-04-23 09:29    2145.70
                     ...   
2019-04-23 14:21    2150.00
2019-04-23 14:31    2150.90
2019-04-23 14:41    2149.65
2019-04-23 14:45    2149.20
2019-04-23 14:47    2149.45
Name: buy, Length: 64, dtype: float64

In [39]:
from lib.charting_lib import *
cache_type = 'backtest_web'
chart_type ='haikin'

symbol='TCS'+'1'
my_cache = cache_state(cache_type)

#######  Hack for freq  ############
my_cache.setValue('TCS','freq','1T')
#####################################
dfohlc = my_cache.getOHLC(symbol)
#print(dfohlc.tail())
#trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')
#render_charts(dfohlc, trade_df, symbol, chart_type)

#web_cache.set(symbol+cache_type+'Trade', tmp_df1.to_json(orient='columns'))
#trade_df = pd.read_json(redis_conn.get(hash_key+cache_type+'Trade'), orient='columns')


freedom_chart(symbol, 'backtest_web')

In [16]:
cache_state('backtest_uni').getValue()


,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,tp,high,last_processed,state,mode,algo,freq,hdf_freq,order_id,job_id
0,TCS,10,1,1,20376.5,0,-11.50000000000091,-0.5646943285048324,17.000000000000455,0.08342944077736832,...,0,2038.0,1554112740.0,SCANNING,full,haikin_1_new,1T,minute,0,1591525970000753-1


In [5]:
my_cache.keys('*backtest_web*')

['done2backtest_web',
 'backtest_web',
 'TCSbacktest_web',
 'TCS1backtest_web',
 'done1backtest_web',
 'BAJFINANCE1backtest_web',
 'last_id_msgbacktest_web',
 'INFRATEL2backtest_web',
 'donebacktest_web',
 'INFRATEL1backtest_web',
 'WIPRObacktest_web',
 'ADANIPORTS2backtest_web',
 'msgBufferQueuebacktest_web',
 'WIPRO1backtest_web',
 'decisionbacktest_web']

In [4]:
for key in my_cache.keys('*backtest_webTrade'):
    my_cache.delete(key)

for key in my_cache.keys('*backtest_webOHLC'):
    my_cache.delete(key)
    

for key in my_cache.keys('*backtest_webTICK'):
    my_cache.delete(key)

In [5]:
def load_algo_to_redis(filename):
    algo = open('algo/'+filename+'.txt')
    algo_str = algo.read()
    my_cache.hset('algos', filename,algo_str)
    
load_algo_to_redis('breakout_1')

## Live Trade Debugging

In [ ]:
live = threadManager('live', ["ohlc_tick_handler"], [ohlc_tick_handler])


In [ ]:
live_cache.setValue('WIPRO','mode','paper')

In [ ]:
live.job.terminate()
live_cache.set('WIPROliveTICK', pd.DataFrame().to_json(orient='columns'))
live_cache.setValue('WIPRO','last_processed',0)
live_cache.getValue('WIPRO')

In [ ]:
#live_cache.getOHLC('WIPRO') 
print(live_cache.xrange('msgBufferQueuelive')[-2:])
print(datetime.fromtimestamp(int(live_cache.xrange('msgBufferQueuelive')[-1][0].split('-')[0] )/1000))


In [ ]:
live_cache.getOHLC('WIPRO').tail()

In [ ]:
my_cache.getOHLC('TCS')

## Quick Backtest

In [ ]:
###########################################################
###                  Quick Backtest                     ###
###########################################################
from talib import MACD, MACDEXT, RSI, BBANDS, MACD, AROON, STOCHF, ATR, OBV, ADOSC, MINUS_DI, PLUS_DI, ADX, EMA, SMA
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR, ROC, MIN, MAX, MINMAX

ohlc_get = lambda df, key: df.iloc[-1][key]
def HAIKINASI(ohlc_data_df):
    REF = lambda key, i: ohlc_get(ohlc_data_df.shift(i), key)
    
    OPEN  = ohlc_data_df['open']
    HIGH  = ohlc_data_df['high']
    LOW   = ohlc_data_df['low']
    CLOSE = ohlc_data_df['close']
    
    haOPEN  = (OPEN.shift(1) + CLOSE.shift(1))/2
    haHIGH  = pd.DataFrame([HIGH,OPEN,CLOSE]).max(axis = 0, skipna = True)
    haLOW   = pd.DataFrame([LOW,OPEN,CLOSE]).min(axis = 0, skipna = True)
    haCLOSE = (OPEN+HIGH+LOW+CLOSE)/4
    
    return (haOPEN, haHIGH, haLOW, haCLOSE)


################ Load Data ###################
from lib.charting_lib import *
cache_type = 'backtest_uni'
symbol='TCS'
my_cache = cache_state(cache_type)

###  Set Freq for Algo
my_cache.setValue('TCS','freq','1T')
dfohlc = my_cache.getOHLC(symbol)
#print(dfohlc.head())

### Load Data
ohlc_data_temp = dfohlc
pd.set_option('display.max_rows', None)
################ Initialize Helper Functions ###################
trade_df1 = pd.DataFrame()
OPEN = ohlc_data_temp['open']
CLOSE = ohlc_data_temp['close']
HIGH = ohlc_data_temp['high']
LOW = ohlc_data_temp['low']
#VOLUME = ohlc_data_temp['volume']

(haOPEN, haHIGH, haLOW, haCLOSE) = HAIKINASI(ohlc_data_temp)

TIME = ohlc_data_temp.index.minute+ohlc_data_temp.index.hour*60

#REF = lambda df, i: df.iloc[-i-1]
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a,b: ROC(a, b) >= 0.1
TREND_DOWN = lambda a,b: ROC(a, b) <= -0.1
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)


def BUY(CLOSE, x, trade_df1):
    #global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["buy"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    return trade_df1

def SELL(CLOSE, x, trade_df1):
    #global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["sell"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    return trade_df1
    
################# Write Algo For Backtest Here ###################

roc = ROC(CLOSE, 10)
bbh, bbm, bbl = BBANDS(CLOSE)

delbb = bbh - bbl

#sell = (REF(roc,0) > 0.1) & (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) < REF(haCLOSE,1))
#buy = (REF(roc,0) < -0.1) & (REF(haOPEN, 0) < REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))
buy = delbb > 2.1 * REF(delbb, 3)
sell = delbb < 0.4 * REF(delbb, 3)

logic = buy | sell

summary_df = trade_df1.copy(deep=True)
#summary_df['haopen'] = haOPEN[ logic ]
#summary_df['haclose'] = haCLOSE[ logic ]
summary_df['roc'] = ROC(CLOSE, 30)[logic]

#print(summary_df.sort_index())
#print(summary_df.sort_index().tail(10))
#print(ROC(CLOSE,3))
#################### Render Charts ###############################

trade_df1 = SELL(CLOSE, sell, trade_df1)
trade_df1 = BUY(CLOSE, buy, trade_df1)

#print(trade_df1.sort_index())
my_cache.setCache(symbol+cache_type+'Trade',trade_df1)
#my_cache.pushTrade(symbol, trade_df1)

chart_type ='haikin'
trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')


render_charts(ohlc_data_temp, trade_df, symbol, chart_type)

In [ ]:

state = 'WAIT'
profit = 0
for index, row in trade_df.sort_index().fillna(0).iterrows():
    if state == 'WAIT':
        if row['sell']!=0:
            #print("S: ->: {}".format(row['sell']))
            state = 'SELL'
            profit = profit + row['sell']
        elif row['buy']!=0:
            #print("B: ->: {}".format(row['buy']))
            state = 'BUY'
            profit = profit - row['buy']
    elif state == 'BUY':
        if row['sell']!=0:
            #print("S: <-: {}".format(row['sell']))
            state = 'WAIT'
            profit = profit + row['sell']
    elif state == 'SELL':
        if row['buy']!=0:
            #print("B: <-: {}".format(row['buy']))
            state = 'WAIT'
            profit = profit - row['buy']
    
if state == 'BUY':
    profit = profit + row['buy']
elif state == 'SELL':
    profit = profit - row['sell']
    
print("Total Profit: {}".format(profit))

In [ ]:
bbh, bbm, bbl = BBANDS(CLOSE)

print(pd.DataFrame(data={'bbl':bbl, 'bbm':bbm, 'bbh':bbh, 'del':(bbh-bbl)/bbl*100}))

In [ ]:
(bbh-bbl)/STDDEV(CLOSE)

In [ ]:
cache_type = 'backtest_web'
symbol='TCS'
my_cache = cache_state(cache_type)
my_cache.getTrades(symbol)

my_cache.get('TCSbacktest_webTrades')

In [ ]:
my_cache.getValue()

In [ ]:
temp_file = pd.HDFStore("data/kite_cache.h5", mode="r")
temp_file.keys()

## Redis Console

In [2]:
from lib.multitasking_lib import *
#from lib.data_model_lib import *
import json
web_cache = cache_state('backtest_web')
live_cache = cache_state('live')
uni_cache = cache_state('backtest_uni')

pd.set_option('display.max_rows', None)

In [3]:
web_cache.hget('algos',algo_name)

NameError: name 'algo_name' is not defined

In [ ]:
web_cache.get('last_id_msgbacktest_web')

In [ ]:
live_cache.hget('algos','breakout_1')

In [ ]:
live_cache.pubsub_channels()

In [ ]:
live_cache.xread({'notificationQueuelivenew':0}, block=2000, count=5000)

In [ ]:
str(web_cache.getOHLC('TCS').index[1].tz_localize(tz='Asia/Calcutta').timestamp())

In [ ]:
web_cache.getValue('WIPRO','last_processed')

In [ ]:
live_cache.remove()

In [ ]:
df = uni_cache.getValue()
df

In [6]:
web_cache.getOHLC('TCS')

,close,high,low,open
2019-04-23 09:15:00,2151.40,2165.00,2145.90,2163.00
2019-04-23 09:16:00,2154.75,2156.30,2150.60,2151.45
2019-04-23 09:17:00,2154.05,2155.15,2153.40,2154.70
2019-04-23 09:18:00,2147.35,2154.95,2147.25,2154.05
2019-04-23 09:19:00,2150.80,2151.00,2146.20,2147.25
2019-04-23 09:20:00,2149.30,2150.80,2147.10,2150.80
2019-04-23 09:21:00,2149.30,2149.90,2148.20,2149.35
2019-04-23 09:22:00,2142.65,2148.65,2140.65,2148.65
2019-04-23 09:23:00,2144.25,2145.95,2141.95,2141.95
2019-04-23 09:24:00,2143.75,2147.30,2143.75,2144.35


In [ ]:
trades_df = live_cache.getTrades('TCS')
trades_df

In [ ]:
(total_profit, max_loss, max_profit, total_win, total_loss, max_winning_streak, max_loosing_streak, trade_log_df) = trade_analysis_raw(trades_df)

In [ ]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from datetime import datetime as dt
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from lib.multitasking_lib import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

### GetData

In [ ]:
logger.setLevel(logging.DEBUG)
stock = 'WIPRO'
fromDate = '2019-4-22'
toDate  = '2019-4-24'
freq = 'minute'
mydate = fromDate
exchange = 'NSE'
ohlc_data = {}
ohlc_data[stock] = getData(stock, fromDate, toDate, exchange, freq, False, stock)

In [ ]:
ohlc_data[stock].head()

# Kite Connect

In [ ]:
from kiteconnect import KiteConnect
kite_api_key = 'b2w0sfnr1zr92nxm'
live_cache.set('KiteAPIKey',kite_api_key)
access_token = live_cache.get('access_token')
print(access_token)
print(live_cache.get('KiteAPIKey'))

kite = KiteConnect(api_key=kite_api_key)
kite.set_access_token(access_token)

In [ ]:
kite.access_token

In [ ]:
kite.profile()

In [ ]:
cancel_order('CIPLA')

In [ ]:
orders_df = pd.DataFrame(kite.orders())
orders_df[['order_id','status','tradingsymbol','transaction_type','quantity','tag','price']]

In [ ]:
live_cache.getValue()

In [ ]:
position_df = pd.DataFrame(kite.positions()['net'])
position_df

In [ ]:
pd.DataFrame(kite.positions()['net']).columns

In [ ]:
orders_df = pd.DataFrame()
for msg in live_cache.xrange('notificationQueuelivenew'):
    tmp_df = pd.DataFrame.from_dict(json.loads(msg[1]['data']), orient='index').transpose()
    orders_df = orders_df.append(tmp_df)
    #print(msg)
orders_df[['order_id', 'status', 'tradingsymbol','order_type','transaction_type','quantity', 'average_price']]

In [ ]:
orders_df.columns

In [ ]:
live_cache.getValue().columns

In [ ]:
kite.margins()

#### Starting Live Trade for a stock

In [ ]:
symbol = 'INFY'
live_cache.add(symbol)
token = int(live_cache.hmget('eq_token',symbol)[0])
live_cache.publish('live_trade_handlerlive', json.dumps({'cmd':'add','value':[token],'mode':'ltp'}))

live_cache.setValue(symbol,'qty','1')
live_cache.setValue(symbol,'SL %','0.4')
live_cache.setValue(symbol,'TP %','1')
live_cache.setValue(symbol,'algo','haikin_1_new')
live_cache.setValue(symbol,'freq','1T')
live_cache.setValue(symbol,'last_processed',datetime.now().timestamp())
live_cache.setValue(symbol,'mode','paper')

In [ ]:
symbol = 'WIPRO'
live_cache.setValue(symbol,'last_processed',datetime.now().timestamp())

In [ ]:
datetime.now().timestamp()

In [ ]:
symbol = 'INFY'
live_cache.setValue(symbol,'mode','PAUSE')

In [ ]:
live_cache.getValue(symbol)

### Kite Ticker Handler Manipulation

In [ ]:
#live_cache.publish('live_trade_handlerlive','profile')
live_cache.publish('live_trade_handlerlive','status')

In [ ]:
live_cache.publish('kite_ticker_handlerlive','INIT')

In [ ]:
live_cache.publish('kite_ticker_handlerlive','CLOSE')

In [ ]:
live_cache.publish('kite_ticker_handlerlive','START')

In [ ]:
live_cache.publish('kite_ticker_handlerlive','profile')

In [ ]:
live_cache.smembers('live_token')

In [ ]:
values = list(map(int,live_cache.smembers('ticker_list')))
values

In [ ]:
live_cache.publish('kite_ticker_handlerlive',json.dumps({'cmd':'add','value':values,'mode':'ltp'}))

In [ ]:
live_cache.publish('kite_ticker_handlerlive',json.dumps({'cmd':'remove','value':[1510401],'mode':'quote'}))


In [ ]:
pd.DataFrame(kite.orders())

#### TickerList Manipulaitons

In [ ]:
live_cache.sadd('ticker_list','2953217')
live_cache.sadd('ticker_list','969473')

live_cache.sadd('ticker_list','1510401')

In [ ]:
values = list(map(int,live_cache.smembers('ticker_list')))
print(values)
live_cache.hmget('eq_token', values)

In [ ]:
live_cache.srem('ticker_list',1510401)

In [ ]:
live_cache.sismember('ticker_list',4268801)

In [ ]:
live_cache.delete('ticker_list')

In [ ]:
live_cache.getValue()

### Get Kite Ticker Status

In [ ]:
## Get Status of Kite
live_cache.get('Kite_Status')

### OrderHandler Monitor and Manipulation

In [ ]:
# Pause Ordering
live_cache.publish('order_handlerlive', 'pause')

In [ ]:
# Resume Ordering
live_cache.publish('order_handlerlive', 'resume')

In [ ]:
# Buy
live_cache.publish('order_handlerlive',json.dumps({'cmd':'buy','symbol':'WIPRO','price':180,'qty':1}))

In [ ]:
# Sell
live_cache.publish('order_handlerlive',json.dumps({'cmd':'sell','symbol':'WIPRO','price':190,'qty':1}))

In [ ]:
# Cancel
live_cache.publish('order_handlerlive',json.dumps({'cmd':'cancel','symbol':'WIPRO'}))

In [ ]:
live_cache.publish('order_handlerlive',json.dumps({'cmd':'getOrders'}))

In [ ]:
## Cancel All the orders
live_cache.publish('order_handlerlive',json.dumps({'cmd':'cancelAll'}))

In [ ]:
#from lib.kite_wrapper_lib import *
cancel_order(kite,['CIPLA'])

In [ ]:
buy_limit('WIPRO',188.5,1)

In [ ]:
sell_limit('WIPRO',188.7,1)

In [ ]:
live_cache.getOHLC('WIPRO').tail(10)

In [ ]:
kite.cancel_order(kite.VARIETY_REGULAR, 200511002924441, parent_order_id=None)

In [ ]:
kite.convert_position(kite.EXCHANGE_NSE, 'WIPRO', transaction_type, position_type, quantity, old_product, new_product)


In [ ]:
kite.modify_order(kite.VARIETY_REGULAR, 200511003019322, parent_order_id=None, quantity=None, price=187.4, order_type=None, trigger_price=None, validity=None, disclosed_quantity=None)


In [ ]:
msg = {'data': '[{"tradable": true, "mode": "quote", "instrument_token": 969473, "last_price": 184.7, "last_quantity": 5, "average_price": 187.07, "volume": 4393242, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 185.0, "high": 189.95, "low": 184.1, "close": 184.0}, "change": 0.38043478260868946}, {"tradable": true, "mode": "quote", "instrument_token": 2953217, "last_price": 1935.3, "last_quantity": 25, "average_price": 1939.47, "volume": 2449395, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 1909.0, "high": 1951.4, "low": 1909.0, "close": 1893.4}, "change": 2.2129502482306886}]'}

In [ ]:
msg['data']

In [ ]:
val = json.loads(msg['data'])

##### Reset state for tradejob

In [ ]:
live_cache.setValue('WIPRO','state','SCANNING')

### Live Cache State Monitor and manipulation

In [ ]:
live_cache.getValue()

In [ ]:
#Modify cache table
stock_key = 'CIPLA'
#live_cache.add(stock_key, True) #TODO: While adding stock to table use True
live_cache.setValue(stock_key, 'algo', 'haikin_1_new')
live_cache.setValue(stock_key, 'freq', '1T')
live_cache.setValue(stock_key, 'qty', 1)
live_cache.setValue(stock_key, 'SL %', 1)
live_cache.setValue(stock_key, 'TP %', 1)
live_cache.setValue(stock_key, 'P&L', 0)
live_cache.setValue(stock_key, 'Total P&L', 0)
live_cache.setValue(stock_key, 'price', 0)
live_cache.setValue(stock_key, 'hdf_freq', 'minute')
live_cache.getValue()

In [ ]:
live_cache.setValue('WIPRO', 'mode', 'paper')

In [ ]:
#live_cache.remove('AXISBANK')
live_cache.getValue()

In [ ]:
live_cache.remove()

#### Get token for Stock symbol

In [ ]:
live_cache.hmget('eq_token',['CIPLA','INFY'])

In [ ]:
live_cache.hmget('eq_token',['TCS','WIPRO','AXISBANK'])

#### Peek into OHLC buffer

In [ ]:
live_cache.getOHLC('INFRATEL').tail(5)

#### msgBufferQueue Monitoring

In [ ]:
peek_msg_queue = live_cache.xrange('msgBufferQueuelive')[-1]
print(peek_msg_queue)
datetime.fromtimestamp(int(peek_msg_queue[0].split('-')[0] )/1000)

In [ ]:
live_cache.xrange('notificationQueuelivenew')

In [ ]:
datetime.fromtimestamp(1589428580968/1000)

In [ ]:
live_cache.getOHLC('INFRATEL').tail()

##### Monitor: OHLC Tick Handler

In [ ]:
val = live_cache.get('last_id_msg')
print(val)
datetime.fromtimestamp(int(val.split('-')[0] )/1000)

In [ ]:
# Timestamp to ID
timestamp_to_id = lambda x:str(int(float(x)*1000))+'-'+str(0)
id_val = timestamp_to_id(live_cache.getValue('WIPRO','last_processed'))
print(id_val)
live_cache.xread({'msgBufferQueuelive':id_val}, count=1)

In [ ]:
# Peek into the last msg processed by tick handler
print(live_cache.getValue('WIPRO','last_processed'))
datetime.fromtimestamp(float(live_cache.getValue('WIPRO','last_processed')))

In [ ]:
## Ticker job monitoring
live_cache.get('tick_count')

In [ ]:
datetime.fromtimestamp(1589535456664/1000)

In [ ]:
msgBufferDump = live_cache.xrange('msgBufferQueuelive')

In [ ]:
fp = open('log/ticker_log.log',mode='w')

In [ ]:
fp.write(json.dumps(msgBufferDump))

### Update Redis with Instrument list

##### Download fresh data and store in cache

In [ ]:
instruments_df = pd.DataFrame(kite.instruments())

In [ ]:
instruments_df.to_hdf('data/instruments.h5',key='instruments')

In [ ]:
instruments_df.columns

In [ ]:
eq_nse = instruments_df.loc[(instruments_df['exchange']=='NSE') & (instruments_df['segment']=='NSE')  & (instruments_df['instrument_type']=='EQ'), ['instrument_token','tradingsymbol']]

In [ ]:
eq_nse.loc[eq_nse['tradingsymbol'].isin(['TCS','WIPRO']),'instrument_token'].values

##### Use cache

In [ ]:
instruments_df = pd.read_hdf('data/instruments.h5',key='instruments')

In [ ]:
instruments_df

In [ ]:
eq_nse = instruments_df.loc[(instruments_df['exchange']=='NSE') & (instruments_df['segment']=='NSE')  & (instruments_df['instrument_type']=='EQ'), ['instrument_token','tradingsymbol']]

In [ ]:
eq_nse.loc[eq_nse['tradingsymbol'].isin(['TCS','WIPRO']),:].values

eq_nse.set_index('instrument_token')

In [ ]:
live_cache.delete('eq_token')

In [ ]:
eq_nse.head().to_dict(orient='records')

In [ ]:
eq_nse.head().set_index('tradingsymbol').transpose().to_dict(orient='records')

In [ ]:
eq_nse.head().set_index('instrument_token').transpose().to_dict(orient='records')[0]

##### Update redis with tokens

In [ ]:
live_cache.hmset('eq_token',eq_nse.set_index('instrument_token').transpose().to_dict(orient='records')[0])
live_cache.hmset('eq_token',eq_nse.set_index('tradingsymbol').transpose().to_dict(orient='records')[0])

In [ ]:
live_cache.hmget('eq_token',['TCS','WIPRO'])

In [ ]:
live_cache.hgetall('eq_token')

In [ ]:
live_cache.hmget('eq_token',[2953217, 969473])

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_hdf('data/kite_cache_day_backup.h5')

In [ ]:
import pandas as pd
kite_cache_path = 'data/kite_cache.h5'
temp_file = pd.HDFStore(kite_cache_path, mode="r")
#rDate = temp_file.get(key).tail(1).index

In [ ]:
temp_file.keys()

In [ ]:
df1 = pd.read_hdf(temp_file, key='/minute/NSE/WIPRO')

In [ ]:
keys = ['BHARTIARTL', '3MINDIA', 'AARTIIND', 'ABB', 'ABFRL', 'ACC', 'ACE', 'ADANIENT', 'ADANIPORTS', 'ADANIPOWER', 'AIAENG', 'AJANTPHARM', 'AKZOINDIA', 'ALBK', 'ALKEM', 'ALLCARGO', 'AMARAJABAT', 'AMBUJACEM', 'ANANTRAJ', 'APOLLOHOSP', 'APOLLOTYRE', 'ARVIND', 'ASAHIINDIA', 'ASHOKLEY', 'ASIANPAINT', 'ASTRAZEN', 'ATFL', 'ATUL', 'AUBANK', 'AUROPHARMA', 'AUTOAXLES', 'AVANTIFEED', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJELEC', 'BAJAJFINSV', 'BAJAJHLDNG', 'BAJFINANCE', 'BALKRISIND', 'BALRAMCHIN', 'BANCOINDIA', 'BANKBARODA', 'BANKBEES', 'BANKINDIA', 'BATAINDIA', 'BBTC', 'BEL', 'BEML', 'BERGEPAINT', 'BFUTILITIE', 'BHARATFIN', 'BHARATFORG', 'BHARTIARTL', 'BHEL', 'BIOCON', 'BLISSGVS', 'BLKASHYAP', 'BLUEDART', 'BOSCHLTD', 'BPCL', 'BRITANNIA', 'BSE', 'CADILAHC', 'CAMLINFINE', 'CANBK', 'CANFINHOME', 'CAPACITE', 'CASTROLIND', 'CDSL', 'CEATLTD', 'CENTRUM', 'CENTURYPLY', 'CENTURYTEX', 'CEREBRAINT', 'CESC', 'CGCL', 'CGPOWER', 'CHENNPETRO', 'CHOLAFIN', 'CIPLA', 'CNOVAPETRO', 'COALINDIA', 'COCHINSHIP', 'COFFEEDAY', 'COLPAL', 'CONCOR', 'COROMANDEL', 'COX&KINGS', 'CRISIL', 'CROMPTON', 'CUB', 'CUMMINSIND', 'CYIENT', 'DAAWAT', 'DABUR', 'DALMIASUG', 'DBCORP', 'DCBBANK', 'DCMSHRIRAM', 'DEN', 'DHAMPURSUG', 'DHFL', 'DIAMONDYD', 'DISHTV', 'DIVISLAB', 'DIXON', 'DLF', 'DMART', 'DREDGECORP', 'DRREDDY', 'ECLERX', 'EICHERMOT', 'EIDPARRY', 'EIHOTEL', 'ELECTCAST', 'EMAMILTD', 'ENDURANCE', 'ENGINERSIN', 'EQUITAS', 'ERIS', 'EROSMEDIA', 'ESCORTS', 'ESTER', 'EXIDEIND', 'FEDERALBNK', 'FEL', 'FINCABLES', 'FORTIS', 'FRETAIL', 'GAIL', 'GATI', 'GDL', 'GEPIL', 'GESHIP', 'GET&D', 'GHCL', 'GICHSGFIN', 'GICRE', 'GILLETTE', 'GLAXO', 'GLENMARK', 'GLOBUSSPR', 'GMBREW', 'GMRINFRA', 'GNA', 'GNFC', 'GODFRYPHLP', 'GODREJAGRO', 'GODREJCP', 'GODREJIND', 'GODREJPROP', 'GOLDBEES', 'GPPL', 'GRANULES', 'GRASIM', 'GREAVESCOT', 'GSFC', 'GSKCONS', 'GSPL', 'GUJALKALI', 'GUJFLUORO', 'GUJGASLTD', 'HATHWAY', 'HAVELLS', 'HBLPOWER', 'HCL-INSYS', 'HCLTECH', 'HDFC', 'HDFCAMC', 'HDFCBANK', 'HDFCLIFE', 'HEIDELBERG', 'HEROMOTOCO', 'HEXAWARE', 'HGS', 'HIKAL', 'HINDALCO', 'HINDCOPPER', 'HINDPETRO', 'HINDUNILVR', 'HINDZINC', 'HONAUT', 'HSCL', 'HSIL', 'HUDCO', 'IBULHSGFIN', 'ICICIBANK', 'ICICIGI', 'ICICINIFTY', 'ICICIPRULI', 'IDBI', 'IDEA', 'IDFC', 'IDFCFIRSTB', 'IEX', 'IFCI', 'IGL', 'INDHOTEL', 'INDIACEM', 'INDIANB', 'INDIGO', 'INDUSINDBK', 'INFIBEAM', 'INFRATEL', 'INFY', 'INOXLEISUR', 'INOXWIND', 'INTELLECT', 'IOB', 'IOC', 'IPCALAB', 'IRB', 'ITC', 'ITDCEM', 'JAGRAN', 'JAICORPLTD', 'JAMNAAUTO', 'JAYAGROGN', 'JAYSREETEA', 'JETAIRWAYS', 'JINDALSTEL', 'JINDWORLD', 'JISLJALEQS', 'JKCEMENT', 'JKPAPER', 'JKTYRE', 'JMFINANCIL', 'JPASSOCIAT', 'JSLHISAR', 'JSWENERGY', 'JSWSTEEL', 'JUBILANT', 'JUBLFOOD', 'JUSTDIAL', 'JYOTHYLAB', 'KAJARIACER', 'KALPATPOWR', 'KANSAINER', 'KARURVYSYA', 'KEC', 'KEI', 'KILITCH', 'KIRIINDUS', 'KOTAKBANK', 'KOTAKNIFTY', 'KSCL', 'KTKBANK', 'L&TFH', 'LALPATHLAB', 'LIBERTSHOE', 'LICHSGFIN', 'LINCOLN', 'LIQUIDBEES', 'LOVABLE', 'LT', 'LUPIN', 'M&M', 'M&MFIN', 'M50', 'MAGMA', 'MAHINDCIE', 'MAHLOG', 'MAHSEAMLES', 'MAJESCO', 'MANALIPETC', 'MANAPPURAM', 'MANGTIMBER', 'MANINDS', 'MANINFRA', 'MARICO', 'MARKSANS', 'MARUTI', 'MASFIN', 'MASTEK', 'MCDOWELL-N', 'MCX', 'MEGH', 'MFSL', 'MGL', 'MIDHANI', 'MINDACORP', 'MINDAIND', 'MINDTREE', 'MMTC', 'MOIL', 'MOLDTKPAC', 'MOREPENLAB', 'MOTHERSUMI', 'MPHASIS', 'MRF', 'MRPL', 'MUKANDLTD', 'MUNJALAU', 'MUTHOOTFIN', 'NATCOPHARM', 'NATIONALUM', 'NAUKRI', 'NAVINFLUOR', 'NAVNETEDUL', 'NBCC', 'NCC', 'NESTLEIND', 'NETWORK18', 'NFL', 'NH', 'NHPC', 'NIACL', 'NIFTY 100', 'NIFTY 200', 'NIFTY 50', 'NIFTY AUTO', 'NIFTY BANK', 'NIFTY COMMODITIES', 'NIFTY ENERGY', 'NIFTY FIN SERVICE', 'NIFTY FMCG', 'NIFTY INFRA', 'NIFTY IT', 'NIFTY JUNIOR', 'NIFTY MEDIA', 'NIFTY METAL', 'NIFTY MIDCAP 100', 'NIFTY MIDCAP 50', 'NIFTY MNC', 'NIFTY PHARMA', 'NIFTY PSU BANK', 'NIFTY REALTY', 'NIFTY SMALL 100', 'NIFTYBEES', 'NIITTECH', 'NLCINDIA', 'NMDC', 'NOCIL', 'NRBBEARING', 'NTPC', 'OBEROIRLTY', 'OFSS', 'OIL', 'OMAXE', 'ONGC', 'ORIENTBANK', 'ORIENTCEM', 'PAGEIND', 'PAPERPROD', 'PARAGMILK', 'PATELENG', 'PCJEWELLER', 'PEL', 'PERSISTENT', 'PETRONET', 'PFC', 'PFIZER', 'PGHH', 'PHILIPCARB', 'PHOENIXLTD', 'PIDILITIND', 'PIIND', 'PNB', 'PNBGILTS', 'PNBHOUSING', 'POLYPLEX', 'POWERGRID', 'PRAJIND', 'PRESTIGE', 'PTC', 'PURVA', 'PVR', 'QUICKHEAL', 'RADICO', 'RADIOCITY', 'RAJESHEXPO', 'RALLIS', 'RAMCOCEM', 'RAMCOIND', 'RAMKY', 'RAYMOND', 'RBLBANK', 'RCF', 'RECLTD', 'RELAXO', 'RELCAPITAL', 'RELIANCE', 'RELINFRA', 'REPCOHOME', 'RICOAUTO', 'RIIL', 'RKFORGE', 'RPOWER', 'SAIL', 'SAKUMA', 'SALASAR', 'SALZERELEC', 'SANGHIIND', 'SANGHVIMOV', 'SANOFI', 'SAREGAMA', 'SBILIFE', 'SBIN', 'SCHAEFFLER', 'SCHAND', 'SCHNEIDER', 'SCI', 'SELAN', 'SEQUENT', 'SHARDAMOTR', 'SHREECEM', 'SHRIRAMCIT', 'SIEMENS', 'SIMPLEXINF', 'SINTEX', 'SIS', 'SJVN', 'SKFINDIA', 'SNOWMAN', 'SOBHA', 'SOLARINDS', 'SOUTHBANK', 'SPARC', 'SPTL', 'SREINFRA', 'SRF', 'SRTRANSFIN', 'STAR', 'STCINDIA', 'STRTECH', 'SUNDARMFIN', 'SUNDRMFAST', 'SUNPHARMA', 'SUNTECK', 'SUNTV', 'SUPREMEIND', 'SUVEN', 'SUZLON', 'SYNDIBANK', 'SYNGENE', 'TAKE', 'TALWALKARS', 'TATACHEM', 'TATACOFFEE', 'TATACOMM', 'TATAELXSI', 'TATAGLOBAL', 'TATAINVEST', 'TATAMOTORS', 'TATAMTRDVR', 'TATAPOWER', 'TATASPONGE', 'TATASTEEL', 'TBZ', 'TCI', 'TCS', 'TECHM', 'TEJASNET', 'TEXMOPIPES', 'TEXRAIL', 'THERMAX', 'THOMASCOOK', 'THYROCARE', 'TIMETECHNO', 'TINPLATE', 'TITAN', 'TNPETRO', 'TNPL', 'TORNTPHARM', 'TORNTPOWER', 'TRENT', 'TRIDENT', 'TRIGYN', 'TRIVENI', 'TTKPRESTIG', 'TV18BRDCST', 'TVSMOTOR', 'TWL', 'UBL', 'UCOBANK', 'UJJIVAN', 'ULTRACEMCO', 'UNIONBANK', 'UNIVCABLES', 'UPL', 'USHAMART', 'VEDL', 'VGUARD', 'VIPCLOTHNG', 'VIPIND', 'VOLTAS', 'VTL', 'WABCOINDIA', 'WALCHANNAG', 'WELCORP', 'WELENT', 'WELSPUNIND', 'WHIRLPOOL', 'WINDMACHIN', 'WIPRO', 'WOCKPHARMA', 'WONDERLA', 'YESBANK', 'ZEEL']

temp_file1 = pd.HDFStore('data/kite_cache_sample.h5', mode="w")
for key in keys:
    hash1 = '/minute/NSE/'+key
    hash2 = '/day/NSE/'+key
    df1 = pd.read_hdf(temp_file, key=hash1)
    df2 = pd.read_hdf(temp_file, key=hash2)
    
    df1.tail(10000).to_hdf(temp_file1, hash1)
    df2.tail(1000).to_hdf(temp_file1, hash2)
    
    #print(key)
    
    temp_file1.close()

In [ ]:
pd.HDFStore('data/kite_cache_sample.h5', mode="r").keys()

In [ ]:
pd.read_hdf('data/kite_cache_sample.h5', key='/minute/NSE/WIPRO')

In [ ]:
temp_file1.close()